# Analisis de informacion de Grupo Interact

##### Conexion a base de datos MySQL para obtener informacion almacenada

In [167]:
import pandas as pd
from sqlalchemy import create_engine

localhost = 'localhost'
user = 'root'
password = 'Mysql12345'
database = 'reportehoras'

# Configurar conexión y probarla

In [168]:
def cargar_tabla(nombre_tabla, con, chunk_size=1000):
    """
    Carga tabla en un DF usando chunks para optimizar la lectura.
    :param nombre_tabla: # Nombre de la tabla
    :param con: # Conexion a la base de datos
    :param chunk_size: # tamano de los pedazos
    :return: Object tipo DataFrame
    """
    return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)

try:
    con = create_engine(f'mysql+pymysql://{user}:{password}@{localhost}/{database}')
    if con is not None:      
        
        tablas = ["clasificaciones", "clientes", "datos_act", "empleados", "modalidades"]
        dfs = {tabla: cargar_tabla(tabla, con) for tabla in tablas}
        
        df_clasificaciones = dfs['clasificaciones']
        df_clientes = dfs['clientes']
        df_datos_act = dfs['datos_act']
        df_empleados = dfs['empleados']
        df_modalidades = dfs['modalidades']
                
except RuntimeError as e:
    print(f'❌ Contraseña incorrecta')
    con = None
except Exception as err:
    print(f"❌ Error de conexión: {err}")
    con = None
finally:
    if con is not None:
        con.dispose()

C:\Users\variz\AppData\Local\Temp\ipykernel_11288\1221055656.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)


##### Realizar merge para obtener df con variables categoricas de llaves foraneas

In [169]:
df_datos_act.head()

,clave_act,numero,clave_cli,clave_mod,clave_cla,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion,bloqueado,tipo,horas,fecha_capt,hora_capt,usuario,baja,tiempo_tot,tiempo_desc
0,1,9011,18,1,22,2024-05-01,07:00,10:00,0.0,0.0,*Reunión para planeación del día con los compa...,1.0,A,None,2024-10-08,10:59:26,9030,0,3.0,3.0
1,2,9011,25,1,45,2024-05-01,10:00,10:30,0.0,0.0,*Soporte a A. Chávez en pólizas entregadas de ...,1.0,A,None,2024-10-08,10:59:26,9030,0,0.5,0.5
2,3,9034,18,1,22,2024-05-01,08:00,09:00,0.0,0.0,capacitación 5s,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
3,4,9034,18,1,22,2024-05-01,09:00,10:00,0.0,0.0,Capacitacion proyecto de reporte de horas,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
4,5,9010,18,1,3,2024-05-01,07:00,16:00,0.0,1.0,"Revisión de temas de red nueva oficina, ubicac...",1.0,A,None,2024-06-11,08:57:08,9034,0,9.0,8.0


In [170]:
# Se revisa parte del df_clasificaciones ya que no permite merge por index 'tipo'
df_clasificaciones.columns
df_clasificaciones.columns = df_clasificaciones.columns.str.strip()
df_clasificaciones.head()

,clave_cla,descripcion,tipo,fecha_capt,hora_capt,usuario,baja
0,1,Soporte,S,2024-04-15,14:34:00,1111,2
1,2,Proyectos,P,2024-04-15,14:35:00,1111,2
2,3,Actividades Administrativas,S,2024-05-01,11:19:40,9010,0
3,4,Actividades Contables,S,2024-05-01,11:19:55,9010,0
4,5,Actualización del Sistema,S,2024-05-01,11:20:15,9010,0


In [171]:
df_concentrado = pd.merge(df_datos_act, df_clientes[['clave_cli', 'nombre']], how='left', on='clave_cli', suffixes=('_datos','_cli'))
df_concentrado = df_concentrado.merge(df_modalidades[['clave_mod','descripcion']], how='left', on='clave_mod', suffixes=('_total','_modalidades'))
df_concentrado = df_concentrado.merge(df_clasificaciones[['clave_cla','descripcion','tipo']], how='left', on='clave_cla', suffixes=('_total', '_clasificaciones'))
df_concentrado = df_concentrado.merge(df_empleados[['numero', 'nombre']], how='left', on='numero', suffixes=('_total', '_empleados'))
df_concentrado.drop(columns=['clave_cli', 'clave_mod', 'clave_cla', 'numero', 'usuario', 'bloqueado', 'horas'], inplace=True)

df_concentrado = df_concentrado[['clave_act', 'nombre_empleados', 'nombre_total', 'descripcion_modalidades', 'descripcion', 'fecha_act', 'hora_ini', 'hora_fin', 'tiempo_t', 'tiempo_p', 'descripcion_total', 'tipo_clasificaciones', 'fecha_capt', 'hora_capt', 'tiempo_tot', 'tiempo_desc']]

df_concentrado.tail()


,clave_act,nombre_empleados,nombre_total,descripcion_modalidades,descripcion,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion_total,tipo_clasificaciones,fecha_capt,hora_capt,tiempo_tot,tiempo_desc
6157,6165,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-09,07:00,17:10,0.0,1.0,Avance en creación de servicio y conexión a FTP,P,2025-01-14,17:17:21,10.17,9.17
6158,6166,JORGE ARIZA RAMIREZ,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-14,07:00,11:00,0.0,0.0,Revisión y corrección de detalles en servicio ...,P,2025-01-14,17:18:07,4.00,4.00
6159,6167,JORGE ARIZA RAMIREZ,GRUPO INTERACT SA DE CV,Presencial Oficina,Juntas y Reuniones,2025-01-14,11:00,12:30,0.0,0.0,Revisión de avances y correcciones con Nataly....,S,2025-01-14,17:19:13,1.50,1.50
6160,6168,JORGE ARIZA RAMIREZ,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-14,12:30,14:00,0.0,0.0,Avance en creación de servicio y conexión a ca...,P,2025-01-14,17:20:48,1.50,1.50
6161,6169,JORGE ARIZA RAMIREZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,Presencial Oficina,Generación de credenciales,2025-01-14,14:00,17:00,0.0,1.0,Impresión de credenciales.,S,2025-01-14,17:21:49,3.00,2.00


In [172]:
# Convertir atributos a tipos de objeto datetime para trabajar con fechas

df_concentrado['fecha_capt'] = pd.to_datetime(df_concentrado['fecha_capt'])
df_concentrado['fecha_act'] = pd.to_datetime(df_concentrado['fecha_act'])
df_concentrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   clave_act                6162 non-null   int64         
 1   nombre_empleados         6162 non-null   object        
 2   nombre_total             6162 non-null   object        
 3   descripcion_modalidades  6162 non-null   object        
 4   descripcion              6162 non-null   object        
 5   fecha_act                6162 non-null   datetime64[ns]
 6   hora_ini                 6162 non-null   object        
 7   hora_fin                 6162 non-null   object        
 8   tiempo_t                 6162 non-null   float64       
 9   tiempo_p                 6162 non-null   float64       
 10  descripcion_total        6162 non-null   object        
 11  tipo_clasificaciones     6162 non-null   object        
 12  fecha_capt               6162 non-

In [173]:
df_concentrado['anio'] = df_concentrado['fecha_act'].dt.year
df_concentrado['mes'] = df_concentrado['fecha_act'].dt.month
resumen = df_concentrado.groupby(['anio', 'mes', 'nombre_empleados', 'nombre_total'])['tiempo_desc'].sum().reset_index()
resumen = resumen.sort_values(by=['anio', 'mes', 'nombre_empleados', 'nombre_total'])

# Mostrar el resultado
resumen

,anio,mes,nombre_empleados,nombre_total,tiempo_desc
0,2024,5,ANA CERVANTES PIEDRAS,ERGON ASFALTOS MEXICO S DE RL DE CV,12.58
1,2024,5,ANA CERVANTES PIEDRAS,ERGON MEXICO S DE RL DE CV,0.98
2,2024,5,ANA CERVANTES PIEDRAS,GRUPO INTERACT SA DE CV,180.99
3,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,GRUPO INTERACT SA DE CV,86.42
4,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,OXIQUIMICA SA DE CV,69.55
...,...,...,...,...,...
497,2025,1,JORGE ARIZA RAMIREZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,2.00
498,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,GRUPO INTERACT SA DE CV,10.00
499,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,54.67
500,2025,1,MIRIAM FABIAN LUNA,GRUPO INTERACT SA DE CV,66.59


In [174]:
resumen.query("anio == 2024 and mes == 9 and nombre_empleados == 'NATALY ALCANTARA HERNANDEZ'")

,anio,mes,nombre_empleados,nombre_total,tiempo_desc
305,2024,9,NATALY ALCANTARA HERNANDEZ,CONDOMINIO CORPORATIVO ANGELOPOLIS AC,1.00
306,2024,9,NATALY ALCANTARA HERNANDEZ,DIVISION CEMMTHOME,6.25
307,2024,9,NATALY ALCANTARA HERNANDEZ,EUWE EUGEN WEXLER DE MEXICO SA DE CV,6.25
308,2024,9,NATALY ALCANTARA HERNANDEZ,GRAMMER AUTOMOTIVE PUEBLA SA DE CV,2.17
309,2024,9,NATALY ALCANTARA HERNANDEZ,GRUPO INTERACT SA DE CV,107.92
310,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,16.33
311,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,41.50


In [175]:
resumen["porcentaje_total"] = resumen.groupby(["anio", "mes", "nombre_empleados"])["tiempo_desc"].transform(lambda x: (x / x.sum())*100)
resumen.query("anio == 2024 and mes == 8 and nombre_empleados == 'NATALY ALCANTARA HERNANDEZ'")
resumen.columns


Index(['anio', 'mes', 'nombre_empleados', 'nombre_total', 'tiempo_desc',
       'porcentaje_total'],
      dtype='object')

In [176]:
resumen = resumen.sort_values(['anio', 'mes', 'nombre_empleados'], ascending=[False, False, False])
resumen

,anio,mes,nombre_empleados,nombre_total,tiempo_desc,porcentaje_total
500,2025,1,MIRIAM FABIAN LUNA,GRUPO INTERACT SA DE CV,66.59,87.629951
501,2025,1,MIRIAM FABIAN LUNA,THYSSENKRUP COMPONENTS GUANAJUATO,9.40,12.370049
498,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,GRUPO INTERACT SA DE CV,10.00,15.463120
499,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,54.67,84.536880
493,2025,1,JORGE ARIZA RAMIREZ,ELECTRONICA CLARION SA DE CV,0.50,0.549451
...,...,...,...,...,...,...
4,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,OXIQUIMICA SA DE CV,69.55,33.463241
5,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,51.87,24.956697
0,2024,5,ANA CERVANTES PIEDRAS,ERGON ASFALTOS MEXICO S DE RL DE CV,12.58,6.466204
1,2024,5,ANA CERVANTES PIEDRAS,ERGON MEXICO S DE RL DE CV,0.98,0.503727


In [177]:
# En este df, se tendra la informacion completa del mes laborado, horas laborados, horas meta, horas permiso, dias de vacaciones
df_info_empleado = resumen.groupby(['anio', 'mes', 'nombre_empleados'])['tiempo_desc'].sum().reset_index()

resumen.rename(columns={'nombre_empleados': 'empleado', 'tiempo_desc': 'tiempo_invertido', 'nombre_total': 'cliente'}, inplace=True)

df_info_empleado.rename(columns={'tiempo_desc': 'horas_efectivas_trabajadas', 'nombre_empleados': 'empleado'}, inplace=True)
df_info_empleado['horas_permiso_personal'] = 0
df_info_empleado['dias_vacaciones'] = 0
df_info_empleado

,anio,mes,empleado,horas_efectivas_trabajadas,horas_permiso_personal,dias_vacaciones
0,2024,5,ANA CERVANTES PIEDRAS,194.55,0,0
1,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,207.84,0,0
2,2024,5,Administrador,0.50,0,0
3,2024,5,BERENICE ROMERO LUNA,237.34,0,0
4,2024,5,DENISSE ORTEGA RAMIREZ,219.03,0,0
...,...,...,...,...,...,...
112,2025,1,BERENICE ROMERO LUNA,10.08,0,0
113,2025,1,IBETH ALMA ESPINOSA VELAZQUEZ,22.00,0,0
114,2025,1,JORGE ARIZA RAMIREZ,91.00,0,0
115,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,64.67,0,0


##### Obtener dias laborables restando dias festivos

In [178]:
import holidays
import calendar

years = range(2022, 2030)

data = []

for year in years:
    mx_holidays = holidays.MX(years=year)
    
    # Inicializar diccionario para almacenar días laborales por mes
    month_workdays = {}
    
    for month in range(1, 13):
        total_days = calendar.monthrange(year, month)[1]
        
        # Días feriados en el mes
        holiday_count = sum(1 for date in mx_holidays if date.year == year and date.month == month)
        
        weekends = sum(1 for day in range(1, total_days + 1) 
                       if calendar.weekday(year, month, day) in [5, 6]) 
        
        workdays = total_days - holiday_count - weekends
        month_workdays[month] = workdays
    
    data.append([year] + list(month_workdays.values()))

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
columns = ["anio"] + [f"{m}" for m in range(1,13)]
df_laborables = pd.DataFrame(data, columns=columns)
df_laborables

,anio,1,2,3,4,5,6,7,8,9,10,11,12
0,2022,20,19,22,21,21,22,21,23,21,21,21,21
1,2023,21,19,22,20,22,22,21,23,20,22,21,20
2,2024,22,20,20,22,22,20,23,22,20,22,20,21
3,2025,22,19,20,22,21,21,23,21,21,23,19,22
4,2026,21,19,21,22,20,22,23,21,21,22,20,22
5,2027,20,19,22,22,20,22,22,22,21,21,21,22
6,2028,20,20,22,20,22,22,21,23,20,22,21,20
7,2029,22,19,21,21,22,21,22,23,19,23,21,20


In [179]:
df_laborables_long = df_laborables.melt(id_vars=['anio'], var_name='mes', value_name='dias_laborales')
df_laborables_long['mes'] = df_laborables_long['mes'].astype(int)

# # Unir df_resultado con df_laborables_long usando anio y mes como claves
print(df_laborables_long.columns)
print(df_info_empleado.columns)
df_final = df_info_empleado.merge(df_laborables_long[['anio', 'mes', 'dias_laborales']], on=['anio', 'mes'], how='left')


# Diferencia de dias_laborales - dias_vacaciones
df_final['dias_laborales'] = df_final['dias_laborales'] - df_final['dias_vacaciones']
 
# Calcular las horas laborales multiplicando días laborales * 9.6
df_final['horas_meta'] = df_final['dias_laborales'] * 9.6
df_final['horas_extra'] = (df_final['horas_efectivas_trabajadas'] - df_final['horas_meta']) - df_final['horas_permiso_personal']
df_final['excedido'] = (df_final['horas_extra'] / df_final['horas_meta']) * 100

# Ordenar df en base a tabla .xlsx
df_final = df_final[['anio', 'mes', 'empleado','dias_laborales', 'horas_efectivas_trabajadas', 'horas_meta', 'horas_extra', 'excedido', 'horas_permiso_personal', 'dias_vacaciones']]
df_final.sort_values(['anio', 'mes', 'empleado'], ascending=[False, True, True])

Index(['anio', 'mes', 'dias_laborales'], dtype='object')
Index(['anio', 'mes', 'empleado', 'horas_efectivas_trabajadas',
       'horas_permiso_personal', 'dias_vacaciones'],
      dtype='object')


,anio,mes,empleado,dias_laborales,horas_efectivas_trabajadas,horas_meta,horas_extra,excedido,horas_permiso_personal,dias_vacaciones
111,2025,1,ANA GABRIEL MARTINEZ BRAVO,22,99.50,211.2,-111.70,-52.888258,0,0
112,2025,1,BERENICE ROMERO LUNA,22,10.08,211.2,-201.12,-95.227273,0,0
113,2025,1,IBETH ALMA ESPINOSA VELAZQUEZ,22,22.00,211.2,-189.20,-89.583333,0,0
114,2025,1,JORGE ARIZA RAMIREZ,22,91.00,211.2,-120.20,-56.912879,0,0
115,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,22,64.67,211.2,-146.53,-69.379735,0,0
...,...,...,...,...,...,...,...,...,...,...
106,2024,12,JOSE PORFIRIO JIMENEZ DIAZ,21,205.23,201.6,3.63,1.800595,0,0
107,2024,12,KASSANDRA MONICA AGUIRRE ZAMORA,21,254.98,201.6,53.38,26.478175,0,0
108,2024,12,MARLENNE RIVERA SANTIAGO,21,153.50,201.6,-48.10,-23.859127,0,0
109,2024,12,MIRIAM FABIAN LUNA,21,234.95,201.6,33.35,16.542659,0,0


In [180]:
df_info_empleado.rename(columns={'nombre_empleado': 'empleado', 'nombre_total': 'cliente', 'tiempo_desc': 'tiempo_invertido', 'porcentaje_total': 'porcentaje_invertido'}, inplace=True)
print(df_info_empleado.columns)
print(df_final.columns)

Index(['anio', 'mes', 'empleado', 'horas_efectivas_trabajadas',
       'horas_permiso_personal', 'dias_vacaciones'],
      dtype='object')
Index(['anio', 'mes', 'empleado', 'dias_laborales',
       'horas_efectivas_trabajadas', 'horas_meta', 'horas_extra', 'excedido',
       'horas_permiso_personal', 'dias_vacaciones'],
      dtype='object')


In [181]:
resumen.query("anio == 2024 and mes == 9 and empleado == 'NATALY ALCANTARA HERNANDEZ'")

,anio,mes,empleado,cliente,tiempo_invertido,porcentaje_total
305,2024,9,NATALY ALCANTARA HERNANDEZ,CONDOMINIO CORPORATIVO ANGELOPOLIS AC,1.00,0.551207
306,2024,9,NATALY ALCANTARA HERNANDEZ,DIVISION CEMMTHOME,6.25,3.445045
307,2024,9,NATALY ALCANTARA HERNANDEZ,EUWE EUGEN WEXLER DE MEXICO SA DE CV,6.25,3.445045
308,2024,9,NATALY ALCANTARA HERNANDEZ,GRAMMER AUTOMOTIVE PUEBLA SA DE CV,2.17,1.196120
309,2024,9,NATALY ALCANTARA HERNANDEZ,GRUPO INTERACT SA DE CV,107.92,59.486275
310,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,16.33,9.001213
311,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,41.50,22.875096


In [193]:
resumen.head()

,anio,mes,empleado,cliente,tiempo_invertido,porcentaje_total
500,2025,1,MIRIAM FABIAN LUNA,GRUPO INTERACT SA DE CV,66.59,87.629951
501,2025,1,MIRIAM FABIAN LUNA,THYSSENKRUP COMPONENTS GUANAJUATO,9.40,12.370049
498,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,GRUPO INTERACT SA DE CV,10.00,15.463120
499,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,54.67,84.536880
493,2025,1,JORGE ARIZA RAMIREZ,ELECTRONICA CLARION SA DE CV,0.50,0.549451


In [216]:
resumen_pivot_empleado = resumen.pivot_table(index=['anio', 'empleado', 'cliente'], columns=['mes'], values='tiempo_invertido', fill_value=0)
resumen_pivot_empresa = resumen.pivot_table(index=['anio', 'cliente', 'empleado'], columns=['mes'], values='tiempo_invertido', fill_value=0)


new_columns = [2, 3, 4]
for column in new_columns:
    resumen_pivot_empleado[column] = 0
    resumen_pivot_empresa[column] = 0
    
resumen_pivot_empleado = resumen_pivot_empleado[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
resumen_pivot_empresa = resumen_pivot_empresa[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

resumen_pivot_empleado.head()

mes                                                                    1   2   \
anio empleado                   cliente                                         
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   0   
                                ERGON MEXICO S DE RL DE CV            0.0   0   
                                GRUPO INTERACT SA DE CV               0.0   0   
                                UNGERER MEXICO S DE RL DE CV          0.0   0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   0   

mes                                                                   3   4   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   0   0   
                                ERGON MEXICO S DE RL DE CV             0   0   
                                GRUPO INTERACT SA DE CV                0   0   
                                UNGERER MEXICO S DE RL DE CV           0   0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                0   0   

mes                                                                       5   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   12.58   
                                ERGON MEXICO S DE RL DE CV              0.98   
                                GRUPO INTERACT SA DE CV               180.99   
                                UNGERER MEXICO S DE RL DE CV            0.00   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                       6   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV    0.00   
                                ERGON MEXICO S DE RL DE CV             13.09   
                                GRUPO INTERACT SA DE CV               159.38   
                                UNGERER MEXICO S DE RL DE CV            7.13   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                       7   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   11.92   
                                ERGON MEXICO S DE RL DE CV              0.00   
                                GRUPO INTERACT SA DE CV               219.37   
                                UNGERER MEXICO S DE RL DE CV            0.00   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                    8   \
anio empleado                   cliente                                     
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   
                                ERGON MEXICO S DE RL DE CV            0.0   
                                GRUPO INTERACT SA DE CV               0.0   
                                UNGERER MEXICO S DE RL DE CV          0.0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   

mes                                                                    9   \
anio empleado                   cliente                                     
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   
                                ERGON MEXICO S DE RL DE CV            0.0   
                                GRUPO INTERACT SA DE CV               0.0   
                                UNGERER MEXICO S DE RL DE CV          0.0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   

mes                                                                    10  \
anio empleado                  

In [218]:
resumen_pivot_empleado.head()

mes                                                                    1   2   \
anio empleado                   cliente                                         
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   0   
                                ERGON MEXICO S DE RL DE CV            0.0   0   
                                GRUPO INTERACT SA DE CV               0.0   0   
                                UNGERER MEXICO S DE RL DE CV          0.0   0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   0   

mes                                                                   3   4   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   0   0   
                                ERGON MEXICO S DE RL DE CV             0   0   
                                GRUPO INTERACT SA DE CV                0   0   
                                UNGERER MEXICO S DE RL DE CV           0   0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                0   0   

mes                                                                       5   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   12.58   
                                ERGON MEXICO S DE RL DE CV              0.98   
                                GRUPO INTERACT SA DE CV               180.99   
                                UNGERER MEXICO S DE RL DE CV            0.00   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                       6   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV    0.00   
                                ERGON MEXICO S DE RL DE CV             13.09   
                                GRUPO INTERACT SA DE CV               159.38   
                                UNGERER MEXICO S DE RL DE CV            7.13   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                       7   \
anio empleado                   cliente                                        
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV   11.92   
                                ERGON MEXICO S DE RL DE CV              0.00   
                                GRUPO INTERACT SA DE CV               219.37   
                                UNGERER MEXICO S DE RL DE CV            0.00   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV                 0.00   

mes                                                                    8   \
anio empleado                   cliente                                     
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   
                                ERGON MEXICO S DE RL DE CV            0.0   
                                GRUPO INTERACT SA DE CV               0.0   
                                UNGERER MEXICO S DE RL DE CV          0.0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   

mes                                                                    9   \
anio empleado                   cliente                                     
2024 ANA CERVANTES PIEDRAS      ERGON ASFALTOS MEXICO  S DE RL DE CV  0.0   
                                ERGON MEXICO S DE RL DE CV            0.0   
                                GRUPO INTERACT SA DE CV               0.0   
                                UNGERER MEXICO S DE RL DE CV          0.0   
     ANA GABRIEL MARTINEZ BRAVO GRUPO INTERACT SA DE CV               0.0   

mes                                                                    10  \
anio empleado                  

In [197]:
df_final.query("anio == 2024 and mes == 7 and empleado == 'ARITZI JANET HERNÁNDEZ HERRERA'")

,anio,mes,empleado,dias_laborales,horas_efectivas_trabajadas,horas_meta,horas_extra,excedido,horas_permiso_personal,dias_vacaciones
32,2024,7,ARITZI JANET HERNÁNDEZ HERRERA,23,203.63,220.8,-17.17,-7.776268,0,0
